In [2]:
%cd /content/drive/MyDrive/hackAI/dataset

/content/drive/MyDrive/hackAI/dataset


In [3]:
!mkdir -p normal. abnormal

In [4]:
!mv N*.wav normal/

mv: target 'normal/' is not a directory


In [5]:
!mv *.wav abnormal/

In [8]:
!python /content/drive/MyDrive/hackAI/main_cross_patients_3.py --data_dir /content/drive/MyDrive/hackAI/dataset

2025-05-25 07:07:29.168760: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748156849.193272   11995 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748156849.200401   11995 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-25 07:07:29.224492: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Using device: cpu
Pre-trained model: facebook/wav2vec2-base
Loading heartbeat dataset...
Loading abnormal samples from /conte

In [9]:
import torch
import torchaudio
from main_cross_patients_3 import HeartbeatLinearProbe, PretrainedAudioEncoder

device= "cuda" if torch.cuda.is_available() else "cpu"
INPUT_DIM = 768
# Load model
checkpoint_path = "/content/dataset/best_heartbeat_classifier.pth"
checkpoint = torch.load(checkpoint_path, map_location=device)
model = HeartbeatLinearProbe(INPUT_DIM)
model.load_state_dict(checkpoint.get('model_state_dict', checkpoint))
model.to(device).eval()



ModuleNotFoundError: No module named 'main_cross_patients_3'

In [108]:
target_length = 16000*5
PATH_AUDIO = "/content/dataset/normal/N_089_sit_Aor.wav"
 # Load audio
waveform, sr = torchaudio.load(PATH_AUDIO )

# Resample if necessary
if sr != 1600:
    resampler = T.Resample(sr, 16000)
    waveform = resampler(waveform)

# Convert to mono if stereo
if waveform.shape[0] > 1:
    waveform = torch.mean(waveform, dim=0, keepdim=True)

# Flatten to 1D
waveform = waveform.squeeze(0)

# Pad or truncate to fixed length
if waveform.shape[0] > target_length :
    # For heartbeat sounds, take center portion to preserve the main heartbeat
    start_idx = (waveform.shape[0] - target_length ) // 2
    waveform = waveform[start_idx:start_idx + target_length ]
else:
    padding = target_length  - waveform.shape[0]
    waveform = torch.nn.functional.pad(waveform, (padding//2, padding - padding//2))

# Normalize audio
if torch.std(waveform) > 0:
    waveform = (waveform - torch.mean(waveform)) / torch.std(waveform)

#return waveform, torch.tensor(self.labels[idx], dtype=torch.long), other_features (torch.tensor)
encoder = PretrainedAudioEncoder()
inputs = encoder(waveform.unsqueeze(0))

Loading pre-trained model: facebook/wav2vec2-base
Encoder weights frozen for linear probing
Feature dimension: 768


In [110]:
output = model(inputs.to(device))


In [111]:
predictions = torch.argmax(output , dim=1)
predictions

tensor([0], device='cuda:0')